# Linear regression PyTorch

In [202]:
import sys
from pathlib import Path
sys.path.append(str(Path("..")))

# imports
import time
from common.utils import get_numeric_data, split

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import torch
from torch.utils.data import DataLoader, TensorDataset

In [203]:
data = get_numeric_data()
data

,Previous qualification (grade),Admission grade,Age at enrollment,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,122.0,127.3,20,0,0,0,0,0.000000,0,0,0,0,0,0.000000,0,10.8,1.4,1.74
1,160.0,142.5,19,0,6,6,6,14.000000,0,0,6,6,6,13.666667,0,13.9,-0.3,0.79
2,122.0,124.8,19,0,6,0,0,0.000000,0,0,6,0,0,0.000000,0,10.8,1.4,1.74
3,122.0,119.6,20,0,6,8,6,13.428571,0,0,6,10,5,12.400000,0,9.4,-0.8,-3.12
4,100.0,141.5,45,0,6,9,5,12.333333,0,0,6,6,6,13.000000,0,13.9,-0.3,0.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,125.0,122.2,19,0,6,7,5,13.600000,0,0,6,8,5,12.666667,0,15.5,2.8,-4.06
4420,120.0,119.0,18,0,6,6,6,12.000000,0,0,6,6,2,11.000000,0,11.1,0.6,2.02
4421,154.0,149.5,30,0,7,8,7,14.912500,0,0,8,9,1,13.500000,0,13.9,-0.3,0.79
4422,180.0,153.8,20,0,5,5,5,13.800000,0,0,5,6,5,12.000000,0,9.4,-0.8,-3.12


In [204]:
X = data.drop("Admission grade", axis=1)
X = StandardScaler().fit_transform(X)       # normalize
X = torch.tensor(X, dtype=torch.float32)
y = data["Admission grade"]
y = torch.tensor(y, dtype=torch.float32)

X_train, X_val, X_test, y_train, y_val, y_test = split(X, y)

In [205]:
class LinearRegression(torch.nn.Module):
    def __init__(self, inputSize):
        super().__init__()
        self.linear = torch.nn.Linear(inputSize, 1)     # linear layer       y = w1*x1 + w2*x2 + ... + b

    def forward(self, x):
        return self.linear(x)

In [206]:
def train(device, X_train, y_train, epochs = 200, batch_size=128, learning_rate=0.001):
    
    start = time.perf_counter()
      
    # WITH COMMENTED WE CAN SEE THE POWER OF GPU
    
    # X_train = X_train.to(device)
    # y_train = y_train.to(device)
    
    # X_train = X_train.to(device)
    # y_train = y_train.to(device).view(-1,1)
    # X_val = X_val.to(device)
    # y_val = y_val.to(device).view(-1,1)
    # X_test = X_test.to(device)
    # y_test = y_test.to(device).view(-1,1)
    
    model = LinearRegression(X_train.shape[1]).to(device)
    criterion = torch.nn.MSELoss()                                          # MSE
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)       # gradient descent implementation
    
    dataset = TensorDataset(X_train, y_train)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for _ in range(epochs):
        for X_batch, y_batch in loader:
            y_pred = model(X_batch)                     # X * W^T + b
            loss = criterion(y_pred, y_batch)           # 1/n ∑ (ytrue - ypred)^2       (MSE)
            
            optimizer.zero_grad()                       # clear gradient from previous steps
            loss.backward()                             # back propagration - calculates gradients of the loss 
            
            optimizer.step()                            # w = w - learning_rate * (∂loss / ∂w)
        
    end = time.perf_counter()
    # print(f"Training time: {end - start:.2f} seconds\n")
    return model, (round(end - start, 2))    

In [207]:
def eval(model, eval_train=False, eval_val=False, eval_test=False,
         X_train=None, y_train=None, X_val=None, y_val=None, X_test=None, y_test=None):
    model.eval()                                    # switch to eval mode
    
    evals = {}
    
    with torch.no_grad():                           # no gradient in testing phasse
        if eval_train:
            y_pred_train = model(X_train).cpu().numpy()
            evals["train"] = mean_squared_error(y_train.cpu().numpy(), y_pred_train)
            
        if eval_val:
            y_pred_val = model(X_val).cpu().numpy()
            evals["val"] = mean_squared_error(y_val.cpu().numpy(), y_pred_val)
        
        if eval_test:
            y_pred_test = model(X_test).cpu().numpy()
            evals["test"] = mean_squared_error(y_test.cpu().numpy(), y_pred_test)
            
    return evals

In [208]:
def look_for_best_params():
    
    CANDIDATE_PARAMS = {
        "epochs": [100, 250, 500, 750],
        "batch_size": [64, 128, 196, 256, 512],
        "learning_rate": [0.001, 0.01]
    }

    device = torch.device("cpu")

    X_train_dev = X_train.to(device)
    y_train_dev = y_train.to(device).view(-1,1)
    X_val_dev = X_val.to(device)
    y_val_dev = y_val.to(device).view(-1,1)

    results = []

    for epochs in CANDIDATE_PARAMS["epochs"]:
        for batch_size in CANDIDATE_PARAMS["batch_size"]:
            for learning_rate in CANDIDATE_PARAMS["learning_rate"]:
                model, seconds = train(device, X_train_dev, y_train_dev, epochs=epochs, batch_size=batch_size, learning_rate=learning_rate)
                evaluation = eval(model, X_val=X_val_dev, y_val=y_val_dev, eval_val=True)["val"]
                results.append({
                    "Time (s)": seconds,
                    "Eval MSE": evaluation,
                    "Epochs": epochs,
                    "Batch size": batch_size,
                    "Learning rate": learning_rate
                })
                print(f"Time: {seconds} s     Eval: {evaluation:.5f}     Epochs: {epochs}     Batch size: {batch_size}      Learning rate: {learning_rate}")
       
# 6-7 mins     
# best:
#   echos - 250
#   batch_size - 512
#   learining_rate - 0.01   
# look_for_best_params()

In [209]:
def print_eval(evaluations: dict):
    print(f"MSE train: {evaluations['train']}")
    print(f"MSE validation: {evaluations['val']}")
    print(f"MSE test: {evaluations['test']}")

CPU

In [210]:
device = torch.device("cpu")

X_train_dev = X_train.to(device)
y_train_dev = y_train.to(device).view(-1,1)
X_val_dev = X_val.to(device)
y_val_dev = y_val.to(device).view(-1,1)
X_test_dev = X_test.to(device)
y_test_dev = y_test.to(device).view(-1,1)

model, seconds = train(device, X_train_dev, y_train_dev, epochs=250, batch_size=512, learning_rate=0.01)
cpu_evaluations = eval(
    model, 
    eval_train=True, X_train=X_train_dev, y_train=y_train_dev, 
    eval_val=True, X_val=X_val_dev, y_val=y_val_dev, 
    eval_test=True, X_test=X_test_dev, y_test=y_test_dev, 
)

print_eval(cpu_evaluations)

cpu_results = cpu_evaluations
cpu_results["time"] = seconds

MSE train: 131.5977325439453
MSE validation: 125.25394439697266
MSE test: 154.86978149414062


GPU

In [211]:
if torch.cuda.is_available():
    device = torch.device("cuda")

    X_train_dev = X_train.to(device)
    y_train_dev = y_train.to(device).view(-1,1)
    X_val_dev = X_val.to(device)
    y_val_dev = y_val.to(device).view(-1,1)
    X_test_dev = X_test.to(device)
    y_test_dev = y_test.to(device).view(-1,1)

    model, seconds = train(device, X_train_dev, y_train_dev, epochs=250, batch_size=512, learning_rate=0.01)
    gpu_evaluations = eval(
        model, 
        eval_train=True, X_train=X_train_dev, y_train=y_train_dev, 
        eval_val=True, X_val=X_val_dev, y_val=y_val_dev, 
        eval_test=True, X_test=X_test_dev, y_test=y_test_dev, 
    )

    print_eval(gpu_evaluations)
else:
    print("Not available")
    gpu_evaluations = {"train": 0, "val": 0, "test": 0}
    
gpu_results = gpu_evaluations
gpu_results["time"] = seconds

MSE train: 131.16177368164062
MSE validation: 124.24231719970703
MSE test: 152.35610961914062


In [212]:
df = pd.DataFrame([cpu_results, gpu_results], index=["CPU", "GPU"])
df

,train,val,test,time
CPU,131.597733,125.253944,154.869781,5.19
GPU,131.161774,124.242317,152.356110,7.09
